# Introduction

In this notebook we'll demonstrate how to use BigDL-Nano to accelerate the LightningLite tool.

### Prepare Environment

Before you start with APIs delivered by BigDL-Nano, you have to make sure BigDL-Nano is correctly installed for PyTorch. If not, please follow [this](../../../../../docs/readthedocs/source/doc/Nano/Overview/nano.md) to setup your environment.

### Load Cifar10 Dataset

Import Cifar10 dataset from torch_vision and modify the train transform. You could access [CIFAR10](https://www.cs.toronto.edu/~kriz/cifar.html) for a view of the whole dataset.

Leveraging OpenCV and libjpeg-turbo, BigDL-Nano can accelerate computer vision data pipelines by providing a drop-in replacement of torch_vision's `datasets` and `transforms`.

In [6]:
from torch.utils.data import DataLoader

from bigdl.nano.pytorch.vision import transforms
from bigdl.nano.pytorch.vision.datasets import CIFAR10

def create_dataloader(data_path, batch_size):
    train_transform = transforms.Compose([
        transforms.Resize(256),
        transforms.ColorJitter(),
        transforms.RandomCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.Resize(128),
        transforms.ToTensor()
    ])

    train_dataset = CIFAR10(root=data_path, train=True,
                            download=True, transform=train_transform)

    train_loader = DataLoader(train_dataset, batch_size=batch_size,
                              shuffle=True, num_workers=0)

    return train_loader

### Custom Model

We use the Resnet18 module but add a Linear layer to change its output size to 10, because the CIFAR10 dataset has 10 classes.

In [7]:
from torch import nn

from bigdl.nano.pytorch.vision.models import vision

class ResNet18(nn.Module):
    def __init__(self, num_classes, pretrained=True, include_top=False, freeze=True):
        super().__init__()
        backbone = vision.resnet18(pretrained=pretrained, include_top=include_top, freeze=freeze)
        output_size = backbone.get_output_size()
        head = nn.Linear(output_size, num_classes)
        self.model = nn.Sequential(backbone, head)

    def forward(self, x):
        return self.model(x)

### Define Train Loop

The `LightningLite` (`bigdl.nano.pytorch.lite.LightningLite`) class is the place where we integrate most optimizations. It extends PyTorch Lightning's `LightningLite` class and has a few more parameters and methods specific to BigDL-Nano.

Our `LightningLite` can be directly used to replace PyTorch Lightning's, all optimizations will be applied automatically, you don't need to change any training codes.

We define the train loop in the overrided `run` method of `LightningLite`, which is required by PyTorch Lightning.

In [8]:
import os
import torch

from bigdl.nano.pytorch.lite import LightningLite


data_path = os.environ.get("DATA_PATH", ".")
batch_size = 256
max_epochs = 3


class Lite(LightningLite):
    def run(self):
        model = ResNet18(10, pretrained=False, include_top=False, freeze=True)
        loss = nn.CrossEntropyLoss()
        optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
        train_loader = create_dataloader(data_path, batch_size)

        model, optimizer = self.setup(model, optimizer)
        train_loader = self.setup_dataloaders(train_loader)
        model.train()

        for _i in range(max_epochs):
            total_loss, num = 0, 0
            for X, y in train_loader:
                optimizer.zero_grad()
                l = loss(model(X), y)
                self.backward(l)
                optimizer.step()
                
                total_loss += l.sum()
                num += 1
            print(f'avg_loss: {total_loss / num}')

### Train in Non-distributed Mode

To run the train loop, we only need to create an instance of `LightningLite` and call its `run` method.

In [9]:
Lite().run()

Files already downloaded and verified
avg_loss: 2.19347882270813
avg_loss: 1.9237079620361328
avg_loss: 1.8780473470687866


Intel Extension for Pytorch (a.k.a [IPEX](https://github.com/intel/intel-extension-for-pytorch)) extends Pytorch with optimizations on intel hardware. BigDL-Nano also integrates IPEX into the `LightningLite`, you can turn on IPEX optimization by setting `use_ipex=True`.

In [10]:
Lite(use_ipex=True).run()

Files already downloaded and verified
avg_loss: 2.1605348587036133
avg_loss: 1.9018423557281494
avg_loss: 1.8672975301742554


### Train in Distributed Mode

You can set the number of processes to enable distributed training to acclerate training. You can also set different distributed strategies, now BigDL-Nano supports 'spawn', 'subprocess' and 'ray', the default strategy is 'subprocess'.

- Note: only the 'subprocess' strategy can be used in interactive environment.

In [11]:
Lite(num_processes=2, strategy="subprocess").run()

Initializing distributed: GLOBAL_RANK: 0, MEMBER: 1/2
Initializing distributed: GLOBAL_RANK: 1, MEMBER: 2/2
----------------------------------------------------------------------------------------------------
distributed_backend=gloo
All distributed processes registered. Starting with 2 processes
----------------------------------------------------------------------------------------------------



Files already downloaded and verified
Files already downloaded and verified


/root/miniconda3/envs/nano-dev/lib/python3.7/site-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)
/root/miniconda3/envs/nano-dev/lib/python3.7/site-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


avg_loss: 2.3727188110351562
avg_loss: 2.3467695713043213
avg_loss: 1.9336925745010376
avg_loss: 1.9219833612442017
avg_loss: 1.8722683191299438avg_loss: 1.875815987586975



Of course you can enable both distributed training and IPEX

In [12]:
Lite(use_ipex=True, num_processes=2, strategy="subprocess").run()

[WARNING] "import intel_pytorch_extension" will be deprecated in future releases. Please use "import torch_ipex" instead.
[WARNING] "import intel_pytorch_extension" will be deprecated in future releases. Please use "import torch_ipex" instead.


Initializing distributed: GLOBAL_RANK: 1, MEMBER: 2/2
Initializing distributed: GLOBAL_RANK: 0, MEMBER: 1/2
----------------------------------------------------------------------------------------------------
distributed_backend=gloo
All distributed processes registered. Starting with 2 processes
----------------------------------------------------------------------------------------------------



Files already downloaded and verified
Files already downloaded and verified


[W LegacyTypeDispatch.h:79] Warning: AutoNonVariableTypeMode is deprecated and will be removed in 1.10 release. For kernel implementations please use AutoDispatchBelowADInplaceOrView instead, If you are looking for a user facing API to enable running your inference-only workload, please use c10::InferenceMode. Using AutoDispatchBelowADInplaceOrView in user code is under risk of producing silent wrong result in some edge cases. See Note [AutoDispatchBelowAutograd] for more details. (function operator())
[W LegacyTypeDispatch.h:79] Warning: AutoNonVariableTypeMode is deprecated and will be removed in 1.10 release. For kernel implementations please use AutoDispatchBelowADInplaceOrView instead, If you are looking for a user facing API to enable running your inference-only workload, please use c10::InferenceMode. Using AutoDispatchBelowADInplaceOrView in user code is under risk of producing silent wrong result in some edge cases. See Note [AutoDispatchBelowAutograd] for more details. (funct

avg_loss: 2.372002363204956
avg_loss: 2.344726085662842
avg_loss: 1.9626035690307617
avg_loss: 1.9547721147537231
avg_loss: 1.906415343284607
avg_loss: 1.9092830419540405
